In [32]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle
import joblib 
import dill 

import mne
from scipy.signal import welch, get_window
from scipy.signal.windows import hamming

from scipy.signal import butter, lfilter, sosfilt, filtfilt, sosfreqz, iirnotch
from scipy import fftpack
from scipy.stats import entropy

In [33]:
base_path_data = '../Data'
base_path_results = '../results'

In [34]:
### TIME SERIES DATA

# dict_keys(['__header__', '__version__', '__globals__', 'SamplingFrequency', 'depth_elecs', 'mni_coords', 'patient_no', 'resected_ch', 'soz_ch', 'spike_24h', 'wake_clip'])
hup_atlas = sio.loadmat(os.path.join(base_path_data, 'HUP_atlas.mat'))
# dict_keys(['__header__', '__version__', '__globals__', 'AgeAtTimeOfStudy', 'ChannelName', 'ChannelPosition', 'ChannelRegion', 'ChannelType', 
# 'Data_N2', 'Data_N3', 'Data_R', 'Data_W', 'FacesLeft', 'FacesRight', 'Gender', 'Hemisphere', 'NodesLeft', 'NodesLeftInflated', 'NodesRegionLeft', 
# 'NodesRegionRight', 'NodesRight', 'NodesRightInflated', 'Patient', 'RegionName', 'SamplingFrequency'])
mni_atlas = sio.loadmat(os.path.join(base_path_data, 'MNI_atlas.mat'))

# (12000, 3431) @ x-axis is time steps, y-axis is electrodes
hup_ts = pd.DataFrame(hup_atlas['wake_clip'])
# (13600, 1765)  @ x-axis is time steps, y-axis is electrodes
mni_ts = pd.DataFrame(mni_atlas['Data_W'])

# accessing columns for patients
hup_patients = pd.DataFrame(hup_atlas['patient_no'])
mni_patients = pd.DataFrame(mni_atlas['Patient'])

# electrode counts
hup_patient_total_el_counts = len(hup_atlas['patient_no'])
mni_patient_total_el_counts = len(mni_atlas['Patient'])

# unique patient ids
hup_patient_ids = np.unique(hup_atlas['patient_no'])
mni_patient_ids = np.unique(mni_atlas['Patient']) 

# sampling frequency
mni_samp_freq = int(mni_atlas['SamplingFrequency'].flatten()[~np.isnan(mni_atlas['SamplingFrequency'].flatten())][0])
hup_samp_freq = int(hup_atlas['SamplingFrequency'].flatten()[~np.isnan(hup_atlas['SamplingFrequency'].flatten())][0])

# mapping electrodes to their respective patients
hup_patient_numbers = hup_atlas['patient_no'].flatten()
hup_el_to_pat_map_dict = {}
for idx, patient_num in enumerate(hup_patient_numbers):
    hup_el_to_pat_map_dict[idx] = patient_num
hup_idx_map_arr = np.array([patient_num for patient_num in hup_patient_numbers]) # arr equivalent

mni_patient_numbers = mni_atlas['Patient'].flatten()
mni_el_to_pat_map_dict = {}
for idx, patient_num in enumerate(mni_patient_numbers):
    mni_el_to_pat_map_dict[idx] = patient_num
mni_idx_map_arr = np.array([patient_num for patient_num in mni_patient_numbers])

## REGION MAPS

dk_atlas_df = pd.read_csv(os.path.join(base_path_data, 'desikanKilliany.csv'))
# columns: Index(['x', 'y', 'z', 'roiNum', 'snum', 'abvr', 'lobe', 'isSideLeft'], dtype='object')
hup_df = pd.read_csv(os.path.join(base_path_data, 'hup_df.csv'))
# columns: Index(['x', 'y', 'z', 'roiNum', 'snum', 'abvr', 'lobe', 'isSideLeft'], dtype='object')
mni_df = pd.read_csv(os.path.join(base_path_data, 'mni_df.csv'))


In [35]:
# already saved
# hup_idx_map_arr_df = pd.DataFrame(hup_idx_map_arr)
# hup_idx_map_arr_df.to_csv(os.path.join(base_path_results, 'hup_idx_map_arr.csv'))
# mni_idx_map_arr_df = pd.DataFrame(mni_idx_map_arr)
# mni_idx_map_arr_df.to_csv(os.path.join(base_path_results, 'mni_idx_map_arr.csv'))

In [36]:
def get_norm_psd(iEEGnormal, data_timeS, sampling_frequency=200):
    """
    Function to compute normalized power spectral densities for different EEG frequency bands.
    
    Args:
    iEEGnormal (DataFrame): A DataFrame to append results to.
    data_timeS (array): Time domain EEG data for a single electrode (1D array)
    sampling_frequency (int): Sampling frequency of the EEG data.
    
    Returns:
    DataFrame: Updated DataFrame with new EEG features.
    """
    
    Fs = sampling_frequency
    window = Fs * 2
    NFFT = window
    
    # Compute PSD
    f, data_psd = welch(data_timeS, fs=Fs, window=hamming(window), 
                       nfft=NFFT, scaling='density', noverlap=window//2)
    
    # filter out noise frequency 57.5Hz to 62.5Hz
    noise_mask = (f >= 57.5) & (f <= 62.5)
    f = f[~noise_mask]
    # Handle 1D data_psd
    data_psd = data_psd[~noise_mask]
    
    def bandpower(psd, freqs, freq_range):
        """Calculate power in the given frequency range."""
        idx = np.logical_and(freqs >= freq_range[0], freqs <= freq_range[1])
        return np.trapezoid(psd[idx], freqs[idx]) 
    
    # Define frequency bands
    bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 13), 
             'beta': (13, 30), 'gamma': (30, 80), 'broad': (1, 80)}
    
    # Calculate band powers (using 1D data_psd)
    band_powers = {band: bandpower(data_psd, f, freq_range) 
                  for band, freq_range in bands.items()}
    
    # Compute log transform
    log_band_powers = {f'{band}log': np.log10(power + 1) 
                      for band, power in band_powers.items()}
    
    # Calculate total power
    total_band_power = np.sum([value for value in log_band_powers.values()])
    
    # Calculate relative powers
    relative_band_powers = {f'{band}Rel': log_band_powers[f'{band}log'] / total_band_power 
                          for band in bands}
    
    # Create DataFrame row
    data_to_append = pd.DataFrame([relative_band_powers])
    # data_to_append['broadlog'] = log_band_powers['broadlog']
    
    # Append to existing DataFrame
    iEEGnormal = pd.concat([iEEGnormal, data_to_append], ignore_index=True)
    
    return iEEGnormal

In [37]:
# not actually used
def compute_shannon_entropy(signal):
    """
    Compute Shannon entropy from EEG signal
    
    Parameters:
    signal: 1D array of EEG values
    
    Returns:
    float: Shannon entropy value
    """
    # 1. Estimate probability distribution using histogram
    hist, bin_edges = np.histogram(signal, bins='auto', density=True)
    
    # 2. Normalize to ensure probabilities sum to 1
    probabilities = hist / hist.sum()
    
    # 3. Remove any zero probabilities (since log(0) is undefined)
    probabilities = probabilities[probabilities > 0]
    
    # 4. Compute Shannon entropy
    H = -np.sum(probabilities * np.log2(probabilities))
    
    # 5. Log transform (optional, for feature scaling)
    H_log = np.log10(H + 1)
    
    return H_log

In [38]:
def get_norm_entropy_full_ts(ieeg_normal, data_time_s, sampling_frequency=200, window_size_mins=1, stride_mins=0.5):
   """
   Compute entropy across full time series using sliding windows.
   
   Args:
       ieeg_normal: DataFrame to append results to
       data_time_s: Time series data
       sampling_frequency: Sampling rate of the data
       window_size_mins: Size of sliding window in minutes
       stride_mins: Stride between windows in minutes
   
   Returns:
       DataFrame with computed entropy values
   """
   if data_time_s.ndim == 1:
       data_time_s = data_time_s.reshape(-1,1)
   
   # Calculate window parameters
   samples_per_window = sampling_frequency * 60 * window_size_mins
   stride_samples = sampling_frequency * 60 * stride_mins
   n_windows = int((len(data_time_s) - samples_per_window) // stride_samples + 1)
   
   entropies = []
   for i in range(n_windows):
       start_idx = int(i * stride_samples)
       end_idx = int(start_idx + samples_per_window)
       window_data = data_time_s[start_idx:end_idx, :]
       
       # Apply filters
       b, a = butter(3, 80/(sampling_frequency/2), btype='low')
       filtered = filtfilt(b, a, window_data.astype(float), axis=0)
       
       b, a = butter(3, 1/(sampling_frequency/2), btype='high')
       filtered = filtfilt(b, a, filtered, axis=0)
       
       b, a = iirnotch(60, 30, sampling_frequency)
       filtered = filtfilt(b, a, filtered, axis=0)
       
       # Compute entropy for this window
       signal = filtered[:, 0]
       
       # Estimate probability distribution
       hist, _ = np.histogram(signal, bins='auto', density=True)
       probabilities = hist / hist.sum()
       probabilities = probabilities[probabilities > 0]
       
       # Compute Shannon entropy
       H = -np.sum(probabilities * np.log2(probabilities))
       entropies.append(H)
   
   # Store raw entropy values
   data_to_append = pd.DataFrame({
       'entropy_fullts': [np.log10(np.mean(entropies) + 1)]  # Log transform of raw entropy values
   })
   
   return pd.concat([ieeg_normal, data_to_append], ignore_index=True)

In [39]:
def get_norm_entropy_1min_seg(ieeg_normal, data_time_s, sampling_frequency=200):
    if data_time_s.ndim == 1:
        data_time_s = data_time_s.reshape(-1,1)
    
    # Get first minute of data
    data_seg = data_time_s[:sampling_frequency*60, :]
    
    # Low pass filter at 80Hz
    b, a = butter(3, 80/(sampling_frequency/2), btype='low')
    data_seg_filtered = filtfilt(b, a, data_seg.astype(float), axis=0)
    
    # High pass filter at 1Hz  
    b, a = butter(3, 1/(sampling_frequency/2), btype='high')
    data_seg_filtered = filtfilt(b, a, data_seg_filtered, axis=0)
    
    # Notch filter at 60Hz
    b, a = iirnotch(60, 30, sampling_frequency)
    data_seg_notch = filtfilt(b, a, data_seg_filtered, axis=0)

    # Compute Shannon entropy for each channel
    data_entropy = np.zeros((data_seg_notch.shape[1], 1))
    for chan in range(data_seg_notch.shape[1]):
        signal = data_seg_notch[:, chan]
        # Estimate probability distribution
        hist, _ = np.histogram(signal, bins='auto', density=True)
        hist = hist / hist.sum()
        data_entropy[chan] = entropy(hist)
    
    # Log transform of non-negative entropy
    data_entropy = np.log10(data_entropy + 1)
    
    # Create new row
    data_to_append = pd.DataFrame({'entropy': data_entropy.flatten()})
    
    return pd.concat([ieeg_normal, data_to_append], ignore_index=True)

In [40]:
hup_psd_features = pd.DataFrame()
hup_entropy_1min_features = pd.DataFrame()
hup_entropy_fullts_features = pd.DataFrame()

# Process HUP data
for patient in hup_patient_ids:
   patient_el_ids = np.where(hup_idx_map_arr == patient)[0]
   
   for idx in patient_el_ids:
       electrode_data = hup_ts.iloc[:, idx].values
       hup_psd_features = get_norm_psd(hup_psd_features, electrode_data)
       hup_entropy_1min_features = get_norm_entropy_1min_seg(hup_entropy_1min_features, electrode_data)
       hup_entropy_fullts_features = get_norm_entropy_full_ts(hup_entropy_fullts_features, electrode_data)

# Combine HUP features
hup_features = pd.concat([
   hup_psd_features,
   hup_entropy_1min_features[['entropy']].rename(columns={'entropy': 'entropy_1min'}),
   hup_entropy_fullts_features[['entropy_fullts']], # Updated column name
], axis=1)

# Initialize MNI DataFrames
mni_psd_features = pd.DataFrame()
mni_entropy_1min_features = pd.DataFrame()
mni_entropy_fullts_features = pd.DataFrame()

# Process MNI data
for patient in mni_patient_ids:
   patient_el_ids = np.where(mni_idx_map_arr == patient)[0]
   
   for idx in patient_el_ids:
       electrode_data = mni_ts.iloc[:, idx].values
       mni_psd_features = get_norm_psd(mni_psd_features, electrode_data)
       mni_entropy_1min_features = get_norm_entropy_1min_seg(mni_entropy_1min_features, electrode_data)
       mni_entropy_fullts_features = get_norm_entropy_full_ts(mni_entropy_fullts_features, electrode_data)

# Combine MNI features
mni_features = pd.concat([
   mni_psd_features,
   mni_entropy_1min_features[['entropy']].rename(columns={'entropy': 'entropy_1min'}),
   mni_entropy_fullts_features[['entropy_fullts']], 
], axis=1)

In [41]:
print("HUP Features shape:", hup_features.shape)
print("HUP Features columns:", hup_features.columns.tolist())
print("\nMNI Features shape:", mni_features.shape)
print("MNI Features columns:", mni_features.columns.tolist())

HUP Features shape: (3431, 8)
HUP Features columns: ['deltaRel', 'thetaRel', 'alphaRel', 'betaRel', 'gammaRel', 'broadRel', 'entropy_1min', 'entropy_fullts']

MNI Features shape: (1765, 8)
MNI Features columns: ['deltaRel', 'thetaRel', 'alphaRel', 'betaRel', 'gammaRel', 'broadRel', 'entropy_1min', 'entropy_fullts']


In [42]:
# Basic shape and info
print("HUP Features:")
print(f"Shape: {hup_features.shape}")
print(f"Columns: {hup_features.columns.tolist()}\n")
print(f"First few rows:\n{hup_features.head()}\n")
print("Basic statistics:\n", hup_features.describe(), "\n")

print("\nMNI Features:")
print(f"Shape: {mni_features.shape}")
print(f"Columns: {mni_features.columns.tolist()}\n")
print(f"First few rows:\n{mni_features.head()}\n")
print("Basic statistics:\n", mni_features.describe(), "\n")

# Check for any missing values
print("Missing values in HUP:", hup_features.isnull().sum())
print("Missing values in MNI:", mni_features.isnull().sum())

# Value ranges for each feature
print("\nHUP Feature Ranges:")
for col in hup_features.columns:
    print(f"{col}: [{hup_features[col].min():.3f}, {hup_features[col].max():.3f}]")

print("\nMNI Feature Ranges:")
for col in mni_features.columns:
    print(f"{col}: [{mni_features[col].min():.3f}, {mni_features[col].max():.3f}]")

HUP Features:
Shape: (3431, 8)
Columns: ['deltaRel', 'thetaRel', 'alphaRel', 'betaRel', 'gammaRel', 'broadRel', 'entropy_1min', 'entropy_fullts']

First few rows:
   deltaRel  thetaRel  alphaRel   betaRel  gammaRel  broadRel  entropy_1min  \
0  0.177924  0.147311  0.135588  0.173211  0.139114  0.226852      0.660497   
1  0.210691  0.231001  0.098005  0.129331  0.048090  0.282882      0.664082   
2  0.248122  0.184206  0.116251  0.126453  0.059346  0.265622      0.662271   
3  0.214907  0.162279  0.114287  0.135578  0.073299  0.299651      0.659488   
4  0.187794  0.182233  0.136686  0.152295  0.069230  0.271761      0.657976   

   entropy_fullts  
0        0.789527  
1        0.793369  
2        0.791429  
3        0.788446  
4        0.786825  

Basic statistics:
           deltaRel     thetaRel     alphaRel      betaRel     gammaRel  \
count  3431.000000  3431.000000  3431.000000  3431.000000  3431.000000   
mean      0.195423     0.172769     0.153454     0.148689     0.098853   


In [43]:
# save combined features (electrode-wise)
# hup_features.to_csv(os.path.join(base_path_results, 'hup_univar_feats_eld.csv'), index=False)
# mni_features.to_csv(os.path.join(base_path_results, 'mni_univar_feats_eld.csv'), index=False)

Aggreggate features based on ROI mappings

In [44]:
def aggregate_features_by_region(features_df, region_df, dk_atlas_df, patient_map_arr):
    """
    Aggregate electrode-level features to region-level features while preserving patient identity
    
    Args:
        features_df: DataFrame with electrode features
        region_df: DataFrame with electrode to region mapping
        dk_atlas_df: DataFrame with region information
        patient_map_arr: Array mapping electrodes to patient IDs
    """
    # Add patient ID and region information to features
    combined_df = pd.concat([
        features_df.reset_index(drop=True),
        pd.DataFrame({
            'roiNum': region_df['roiNum'].reset_index(drop=True),
            'patient_id': patient_map_arr
        })
    ], axis=1)
    
    # Define features to aggregate
    spectral_features = ['deltaRel', 'thetaRel', 'alphaRel', 'betaRel', 'gammaRel']
    entropy_features = ['entropy_1min', 'entropy_fullts']
    
    # Group by both patient and region
    region_features = []
    
    for (pat_id, roi), group in combined_df.groupby(['patient_id', 'roiNum']):
        row_dict = {
            'patient_id': pat_id,
            'roiNum': roi
        }
        
        # Calculate means for each feature
        for feat in spectral_features + entropy_features:
            if feat in group.columns:
                row_dict[f"{feat}_mean"] = group[feat].mean()
        
        region_features.append(row_dict)
    
    # Convert to DataFrame
    region_features_df = pd.DataFrame(region_features)
    
    # Add region names
    region_features_df = pd.merge(
        region_features_df,
        dk_atlas_df[['roiNum', 'roi']].drop_duplicates('roiNum'),
        on='roiNum',
        how='left'
    )
    
    return region_features_df

In [45]:
def aggregate_and_average_by_region(features_df, region_df, dk_atlas_df, patient_map_arr):
    """
    Average features across all electrodes in each region
    Arguments added for region name mapping and patient tracking
    """
    combined_df = pd.concat([
        features_df.reset_index(drop=True),
        pd.DataFrame({
            'roiNum': region_df['roiNum'].reset_index(drop=True),
            'patient_id': patient_map_arr
        })
    ], axis=1)
    
    # First average within patient-region
    patient_region_avg = combined_df.groupby(['patient_id', 'roiNum']).mean()
    
    # Then average across patients for each region
    region_avg = patient_region_avg.groupby('roiNum').mean()
    
    # Add region names
    region_avg = pd.merge(
        region_avg,
        dk_atlas_df[['roiNum', 'roi']].drop_duplicates('roiNum'),
        on='roiNum',
        how='left'
    )
    
    return region_avg

In [46]:
hup_univar_feats = hup_features
mni_univar_feats = mni_features

# Process HUP data
hup_region_features = aggregate_features_by_region(
    hup_univar_feats, 
    hup_df, 
    dk_atlas_df, 
    hup_idx_map_arr
)

hup_region_avg_feats = aggregate_and_average_by_region(
    hup_univar_feats, 
    hup_df, 
    dk_atlas_df, 
    hup_idx_map_arr
)

# Process MNI data
mni_region_features = aggregate_features_by_region(
    mni_univar_feats, 
    mni_df, 
    dk_atlas_df, 
    mni_idx_map_arr
)

mni_region_avg_feats = aggregate_and_average_by_region(
    mni_univar_feats, 
    mni_df, 
    dk_atlas_df, 
    mni_idx_map_arr
)

In [47]:
# print all the features stored in hup_region_features
print(hup_region_features.columns)

Index(['patient_id', 'roiNum', 'deltaRel_mean', 'thetaRel_mean',
       'alphaRel_mean', 'betaRel_mean', 'gammaRel_mean', 'entropy_1min_mean',
       'entropy_fullts_mean', 'roi'],
      dtype='object')


In [48]:
# Verify HUP data
print("HUP Dataset Verification:")
print("-----------------------")
print(f"Total number of rows: {len(hup_region_features)}")
print(f"Unique patients: {len(hup_region_features['patient_id'].unique())}")
print(f"Unique regions: {len(hup_region_features['roi'].unique())}")

# Count regions per patient
hup_regions_per_patient = hup_region_features.groupby('patient_id')['roi'].count()
print("\nNumber of regions per HUP patient:")
print(hup_regions_per_patient.describe())
print("\nExample - Patient with most regions:")
max_patient = hup_regions_per_patient.idxmax()
print(f"\nPatient {max_patient} regions:")
print(hup_region_features[hup_region_features['patient_id'] == max_patient][['roi', 'deltaRel_mean', 'entropy_1min_mean']])

# Verify MNI data
print("\nMNI Dataset Verification:")
print("-----------------------")
print(f"Total number of rows: {len(mni_region_features)}")
print(f"Unique patients: {len(mni_region_features['patient_id'].unique())}")
print(f"Unique regions: {len(mni_region_features['roi'].unique())}")

# Count regions per patient
mni_regions_per_patient = mni_region_features.groupby('patient_id')['roi'].count()
print("\nNumber of regions per MNI patient:")
print(mni_regions_per_patient.describe())

print("\nExample - Hippocampus across HUP patients:")
hippocampus_data = hup_region_features[hup_region_features['roi'] == 'Left-Hippocampus']
print(hippocampus_data[['patient_id', 'deltaRel_mean', 'entropy_1min_mean']])

print("\nFeature availability check:")
for column in hup_region_features.columns:
    if column.endswith('_mean'):
        non_null = hup_region_features[column].notna().sum()
        print(f"{column}: {non_null} non-null values")

HUP Dataset Verification:
-----------------------
Total number of rows: 1064
Unique patients: 60
Unique regions: 76

Number of regions per HUP patient:
count    60.000000
mean     17.733333
std       5.724958
min       5.000000
25%      14.000000
50%      18.000000
75%      21.000000
max      32.000000
Name: roi, dtype: float64

Example - Patient with most regions:

Patient 48 regions:
                             roi  deltaRel_mean  entropy_1min_mean
784             Left-Hippocampus       0.189761           0.662787
785                Left-Amygdala       0.209100           0.659783
786            Right-Hippocampus       0.206738           0.661654
787              ctx-lh-bankssts       0.180409           0.665927
788   ctx-lh-caudalmiddlefrontal       0.160676           0.664392
789            ctx-lh-entorhinal       0.191868           0.664203
790              ctx-lh-fusiform       0.181073           0.665023
791      ctx-lh-inferiortemporal       0.191604           0.668215
792  ctx

In [49]:
# # save combined features (region-wise)
# hup_region_features.to_csv(os.path.join(base_path_results, 'hup_univar_feats_reg.csv'), index=False)
# mni_region_features.to_csv(os.path.join(base_path_results, 'mni_univar_feats_reg.csv'), index=False)

# Save electrode-level features
hup_univar_feats.to_csv(os.path.join(base_path_results, 'hup_univar_feats_raw.csv'), index=True) 
mni_univar_feats.to_csv(os.path.join(base_path_results, 'mni_univar_feats_raw.csv'), index=True)

In [50]:
# hup_region_avg_feats.to_csv(os.path.join(base_path_results, 'hup_univar_feats_reg_avg.csv'), index=False) # already saved
# mni_region_avg_feats.to_csv(os.path.join(base_path_results, 'mni_univar_feats_reg_avg.csv'), index=False)

In [51]:
hup_region_avg_feats.shape, mni_region_avg_feats.shape

((76, 10), (68, 10))

In [52]:
# print the dimensions of hup_region_features
print(hup_region_features.shape)
print(mni_region_features.shape)

(1064, 10)
(639, 10)
